## MERRA2 Aerosol Hourly Granularity
- Download
- Trim
- Create Cutout
- Generate Wind Outputs

# 1. Imports and Download

In [ ]:
import geodata
import xarray as xr
import logging
import matplotlib.pyplot as plt
logging.basicConfig(level=logging.INFO)

In [ ]:
DS = geodata.Dataset(module="merra2",
					 years=slice(2015, 2015),
					 months=slice(2,2),
                     weather_data_config = "surface_aerosol_hourly")

In [ ]:
if DS_hourly.prepared == False:
	DS_hourly.get_data()

## 2. Trim

In [ ]:
## Variables before trimming
ds = xr.open_dataset(DS_hourly.downloadedFiles[0][1])
list(ds.data_vars)

In [ ]:
DS.trim_variables()

In [ ]:
## Variables after trimming
ds = xr.open_dataset(DS.downloadedFiles[0][1])
list(ds.data_vars)

# 3. Create Cutout

In [ ]:
cutout = geodata.Cutout(name="beijing-2015-pm25-8",
                        module="merra2",
                        weather_data_config="surface_aerosol_hourly",
                        xs=slice(110, 125),
                        ys=slice(35, 45),
                        years=slice(2015, 2015),
                        months=slice(2,2))


In [ ]:
cutout.prepare(overwrite=True)

In [ ]:
## Verify Cutout Coords
cutout.coords

# 4. Generate Outputs

In [ ]:
ds_pm25 = geodata.convert.pm25(cutout)
df_pm25 = ds_pm25.to_dataframe(name='pm25')
df_pm25.reset_index(inplace=True)
df_pm25

In [ ]:
df_pm25.to_csv('beijing-2015-pm25.csv')

# 5. Plot Outputs

In [ ]:
## using panda plots
df_pm25[(df_pm25.lat == 39.5) & (df_pm25.lon == 116.25)].plot(x='time', y='pm25')
plt.show()

In [ ]:
## using xarray plots (http://xarray.pydata.org/en/stable/plotting.html)
ds_pm25.isel(y=1).plot.line(x='time', hue='lon');

In [ ]:
ds_pm25.isel(time=1).plot();

In [ ]:
# plot with cartopy
import cartopy.crs as ccrs

In [ ]:
ax = plt.axes(projection=ccrs.Miller(116.25))
ax.set_extent([110, 125, 35, 45])
ds_pm25.isel(time=0).plot.contourf(ax=ax, transform=ccrs.PlateCarree());
ax.coastlines();

In [ ]:
ax = plt.subplot(projection=ccrs.PlateCarree());
ax.set_extent([115, 122, 37, 43])
ds_pm25.isel(time=0).plot.pcolormesh('lon', 'lat', ax=ax);
ax.coastlines();